In [2]:
import subprocess
def c_import(library, elements=None, name=None, always_reimport= True, always_reinstall = True):
  if elements:
    import_str = f'from {library} import {", ".join(elements)}'
    tested_install_var = ", ".join(elements)
    any_not_installed = True if any(e not in globals() for e in elements) else False
  else:
    import_str = f'import {library}'
    tested_install_var = library
    any_not_installed = True if library not in globals() else False
  if name:
    import_str = f'{import_str} as {name}'
    tested_install_var = name
    any_not_installed = True if name not in globals() else False

  def sub_install():
    subprocess.run(f'pip install {library}', shell=True, check=True)
    print(f'Library {library} installed successfully.')

  def sub_import():
    exec(import_str, globals())
    print(f'Library {library} imported successfully. As: \n {import_str}')

  if always_reinstall:
    try:
      sub_install()
      sub_import()
    except subprocess.CalledProcessError:
      print(f'Failed to install {library}.')
    except ImportError as err:
      print(f'After Install. Import error: {err}')

  else:
    if always_reimport == True or any_not_installed == True:
        try:
          sub_import()
        except ImportError as err:
          print(f'Import error: {err}')
          if library in str(err):
            try:
                # Use subprocess to run the pip install command
                sub_install()
                sub_import()
            except subprocess.CalledProcessError:
                print(f'Failed to install {library}.')
    else:
      print(f'"{tested_install_var}" already installed and imported')

In [3]:
import_config = {'always_reimport': False, 'always_reinstall': False}
c_import('pandas',name='pd',**import_config)
c_import('csv',**import_config)
c_import('json',**import_config)
c_import('os',**import_config)
c_import('subprocess',**import_config)
c_import('tqdm.notebook',['tqdm'],**import_config)
c_import('concurrent.futures',['ProcessPoolExecutor'],**import_config)
c_import('datetime',**import_config)
c_import('argparse', **import_config)
c_import('codecs', **import_config)
c_import('os', **import_config)
c_import('sys', **import_config)
c_import('numpy',name='np', **import_config)

Library pandas imported successfully. As: 
 import pandas as pd
Library csv imported successfully. As: 
 import csv
Library json imported successfully. As: 
 import json
Library os imported successfully. As: 
 import os
"subprocess" already installed and imported
Library tqdm.notebook imported successfully. As: 
 from tqdm.notebook import tqdm
Library concurrent.futures imported successfully. As: 
 from concurrent.futures import ProcessPoolExecutor
Library datetime imported successfully. As: 
 import datetime
Library argparse imported successfully. As: 
 import argparse
Library codecs imported successfully. As: 
 import codecs
"os" already installed and imported
Library sys imported successfully. As: 
 import sys
Library numpy imported successfully. As: 
 import numpy as np


In [5]:
input_dir = "../input/reddit_input/"
output_dir = "../output/reddit_output/"
# split_files_dir = input_dir+'split_data/' #split JSONs output
liwcdic_file_dir = '../input/Spanish_LIWC2007_Dictionary.dic'
# Specify the folder path where you want to search for .zst files
zst_files_dir = '/Volumes/Drakôn Kholkikos - 2TB/Tesis-Grado/input/reddit_input/'

## v3

In [ ]:
reddit_files_titles = [
  # 'argentina_submissions',
  'argentina_comments',
  # 'Republica_Argentina_submissions',
  # 'Republica_Argentina_comments',
  # 'RepublicaArgentina_submissions',
  # 'RepublicaArgentina_comments',
]

In [7]:
class liwc:

    def load_liwc_dict(self, liwcdic_file):
        file_content = codecs.open(liwcdic_file, "r", "utf-8").read()
        cate_text = file_content[file_content.find("%")+1:file_content[1:].find("%")].strip()
        for line in cate_text.split("\n"):
            self.liwc_cate_name_by_number[int(line.strip().split("\t")[0])] = line.strip().split("\t")[1]

        dict_text = file_content[file_content[1:].find("%")+2:].strip()
        for line in dict_text.split("\n"):
            self.liwc_cate_number_by_word[line.strip().split("\t")[0]] = set([int(item) for item in line.strip().split("\t")[1:]])

    def __init__(self, liwcdic_file=liwcdic_file_dir):

        self.liwc_category_names = ["WC",'Funct', 'TotPron', 'PronPer', 'Yo', 'Nosotro', 'TuUtd', 'ElElla', 'Ellos', 'PronImp', 'Articulo', 'Verbos', 'VerbAux', 'Pasado', 'Present', 'Futuro', 'Adverb', 'Prepos', 'Conjunc', 'Negacio', 'Cuantif', 'Numeros', 'Maldec', 'verbYO', 'verbTU', 'verbNOS', 'verbosEL', 'verbELLOS', 'Subjuntiv', 'VosUtds', 'formal', 'informal', 'verbVos', 'Social', 'Familia', 'Amigos', 'Humanos', 'Afect', 'EmoPos', 'EmoNeg', 'Ansiedad', 'Enfado', 'Triste', 'MecCog', 'Insight', 'Causa', 'Discrep', 'Tentat', 'Certeza', 'Inhib', 'Incl', 'Excl', 'Percept', 'Ver', 'Oir', 'Sentir', 'Biolog', 'Cuerpo', 'Salud', 'Sexual', 'Ingerir', 'Relativ', 'Movim', 'Espacio', 'Tiempo', 'Trabajo', 'Logro', 'Placer', 'Hogar', 'Dinero', 'Relig', 'Muerte', 'Asentir', 'NoFluen', 'Relleno']
        self.liwc_cate_name_by_number = {}
        self.liwc_cate_number_by_word = {}

        if os.path.exists(liwcdic_file) == False:

            sys.exit()
        else:
            self.load_liwc_dict(liwcdic_file)

    def getLIWCCount(self, text):
        count_by_categories = {"WC":0,'Funct': 0, 'TotPron': 0, 'PronPer': 0, 'Yo': 0, 'Nosotro': 0, 'TuUtd': 0, 'ElElla': 0, 'Ellos': 0, 'PronImp': 0, 'Articulo': 0, 'Verbos': 0, 'VerbAux': 0, 'Pasado': 0, 'Present': 0, 'Futuro': 0, 'Adverb': 0, 'Prepos': 0, 'Conjunc': 0, 'Negacio': 0, 'Cuantif': 0, 'Numeros': 0, 'Maldec': 0, 'verbYO': 0, 'verbTU': 0, 'verbNOS': 0, 'verbosEL': 0, 'verbELLOS': 0, 'Subjuntiv': 0, 'VosUtds': 0, 'formal': 0, 'informal': 0, 'verbVos': 0, 'Social': 0, 'Familia': 0, 'Amigos': 0, 'Humanos': 0, 'Afect': 0, 'EmoPos': 0, 'EmoNeg': 0, 'Ansiedad': 0, 'Enfado': 0, 'Triste': 0, 'MecCog': 0, 'Insight': 0, 'Causa': 0, 'Discrep': 0, 'Tentat': 0, 'Certeza': 0, 'Inhib': 0, 'Incl': 0, 'Excl': 0, 'Percept': 0, 'Ver': 0, 'Oir': 0, 'Sentir': 0, 'Biolog': 0, 'Cuerpo': 0, 'Salud': 0, 'Sexual': 0, 'Ingerir': 0, 'Relativ': 0, 'Movim': 0, 'Espacio': 0, 'Tiempo': 0, 'Trabajo': 0, 'Logro': 0, 'Placer': 0, 'Hogar': 0, 'Dinero': 0, 'Relig': 0, 'Muerte': 0, 'Asentir': 0, 'NoFluen': 0, 'Relleno': 0}

        count_by_categories["WC"] = len(text.split())

        for word in text.split():

            cate_numbers_word_belongs = set([])
            if word in self.liwc_cate_number_by_word:
                cate_numbers_word_belongs = self.liwc_cate_number_by_word[word]

            else:

                #liwc words have *. eg: balcon*
                word = word[:-1]
                while len(word) > 0:
                    if (word+"*") in self.liwc_cate_number_by_word:
                        cate_numbers_word_belongs = self.liwc_cate_number_by_word[word+"*"]
                        break
                    else:
                        word = word[:-1]

            for num in cate_numbers_word_belongs:
                count_by_categories[self.liwc_cate_name_by_number[num]] += 1

        return count_by_categories

In [11]:
def process_chunk(df):
    def remove_invalids(col='', df=''):
        invalid_options = [np.nan, 'nan', None, 0, '0', 'NaN', '[deleted]', '[removed]']
        df = df.dropna(subset=[col], how='all')
        df = df[~df[col].isin(invalid_options)]
        return df

    df = remove_invalids('text', df)

    if 'title' in df.columns: #if its a submission (has title)
        #remove invalid/incomplete obs
        df = remove_invalids('title', df)
        #join title and text (yes fillna just in case something passed previous cleanup)
        df['text'] = df['title'].fillna('') + '\n' + df['text'].fillna('')
        #now drop it
        df = df.drop(columns=['title'])
    return df

In [13]:
# Define the path to your large CSV file
large_csv_file = '/Volumes/Drakôn Kholkikos - 2TB/Tesis-Grado/output/reddit_output/argentina_comments-output_table.csv'
source = 'argentina_comments'
source_output_dir = f'../output/reddit_output/filtered_tables_LIWC_count/comments'

# Specify the chunk size (number of rows per chunk)
chunk_size = 100000  # Adjust this value according to your memory capacity

# Initialize a counter to keep track of the chunk number
chunk_number = 0

# Create a CSV reader object
csv_reader = pd.read_csv(large_csv_file, sep=',', chunksize=chunk_size)

# Iterate through the CSV in chunks
for chunk in csv_reader:
    # Process the chunk as needed (e.g., data manipulation, filtering, etc.)
    chunk = process_chunk(chunk)

    tqdm.pandas(desc = f'Computing {chunk_number} LIWC')
    # Assuming you have the LIWC counts as a Series
    liwc_counts = chunk['text'].progress_apply(liwc().getLIWCCount)
    liwc_df = pd.DataFrame(liwc_counts.tolist())

    # Now the DataFrame will have columns for each LIWC category
    df_merge = pd.concat([chunk, liwc_df], axis=1)

    # You can save or process each chunk here
    chunk_number += 1
    output_file = f'{source_output_dir}/chunk_liwc_{chunk_number}.csv'
    df_merge.to_csv(output_file, index=False, sep=',', quoting=csv.QUOTE_ALL, escapechar='\\')  # Save the chunk to a new CSV file


Computing 0 LIWC:   0%|          | 0/93964 [00:00<?, ?it/s]

Computing 1 LIWC:   0%|          | 0/94489 [00:00<?, ?it/s]

Computing 2 LIWC:   0%|          | 0/94899 [00:00<?, ?it/s]

Computing 3 LIWC:   0%|          | 0/95561 [00:00<?, ?it/s]

Computing 4 LIWC:   0%|          | 0/94707 [00:00<?, ?it/s]

...

Computing 92 LIWC:   0%|          | 0/94394 [00:00<?, ?it/s]

Computing 93 LIWC:   0%|          | 0/94924 [00:00<?, ?it/s]

Computing 94 LIWC:   0%|          | 0/94587 [00:00<?, ?it/s]

Computing 95 LIWC:   0%|          | 0/95019 [00:00<?, ?it/s]

Computing 96 LIWC:   0%|          | 0/65242 [00:00<?, ?it/s]

In [19]:
import os
import pandas as pd

# Specify your directory containing CSV chunks
directory = '/Volumes/Drakôn Kholkikos - 2TB/Tesis-Grado/output/reddit_output/filtered_tables_LIWC_count/comments'

# Get a list of all files in the directory
files = os.listdir(directory)

# Filter for CSV files
csv_files = [file for file in files if file.endswith('.csv')]

# Initialize an empty DataFrame to store the merged data
merged_df = pd.DataFrame()

# Iterate through CSV files and merge them
for csv_file in tqdm(csv_files):
    file_path = os.path.join(directory, csv_file)
    chunk_df = pd.read_csv(file_path, sep=',')
    merged_df = pd.concat([merged_df, chunk_df], ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('/Volumes/Drakôn Kholkikos - 2TB/Tesis-Grado/output/reddit_output/filtered_tables_LIWC_count/merged.csv', index=False)


  0%|          | 0/97 [00:00<?, ?it/s]

/var/folders/fv/m61tb7rx6n5_86nwkz1zl_300000gn/T/ipykernel_30332/2805388866.py:19: DtypeWarning: Columns (0,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  chunk_df = pd.read_csv(file_path, sep=',')
/var/folders/fv/m61tb7rx6n5_86nwkz1zl_300000gn/T/ipykernel_30332/2805388866.py:19: DtypeWarning: Columns (0,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  chunk_df = pd.read_csv(file_path, sep=',')
/var/folders/fv/m61tb7rx6n5_86nwkz1zl_300000gn/T/ipykernel_30332/2805388866.py:19: DtypeWarning: Columns (0,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  chunk_df = pd.read_csv(file_path, sep=',')
/var/folders/fv/m61tb7rx6n5_86nwkz1zl_300000gn/T/ipykernel_30332/2805388866.py:19: DtypeWarning: Columns (0,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  chunk_df = pd.read_csv(file_path, sep=',')
/var/folders/fv/m61tb7rx6n5_86nwkz1zl_300000gn/T/ipykernel_30332/2805388

In [21]:
LIWC_keys = list(
  merged_df.columns[
    list(merged_df.columns).index('permalink') + 1 #Gets the last column before LIWC count
    :  ]
  )


In [23]:
for key in LIWC_keys:
    merged_df[key] = pd.to_numeric(merged_df[key], errors='coerce')

In [24]:
result_df = merged_df.groupby(['year', 'month', 'day'])[LIWC_keys].mean().reset_index()

In [25]:
# Calculate the count of rows for each date
count_df = merged_df.groupby(['year', 'month', 'day'])['id'].count().reset_index()
count_df.rename(columns={'id': 'Obs'}, inplace=True)

In [26]:
# Merge the average data with the count data
result_df = pd.merge(result_df, count_df, on=['year', 'month', 'day'])


In [27]:
result_df.head(10)

,year,month,day,WC,Funct,TotPron,PronPer,Yo,Nosotro,TuUtd,...,Logro,Placer,Hogar,Dinero,Relig,Muerte,Asentir,NoFluen,Relleno,Obs
0,2017.0,12.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167
1,2018.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,932
2,2018.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2694
3,2018.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2615
4,2018.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2661
5,2018.0,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2637
6,2018.0,1.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1817
7,2018.0,1.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1978
8,2018.0,1.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3113
9,2018.0,1.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3381


In [ ]:

# Save the results to a new DataFrame
# result_df.to_csv('averages_by_date_with_count.csv', index=False)
result_df.to_csv('comments_liwc_average.csv', index=False, sep=',', quoting=csv.QUOTE_ALL, escapechar='\\')

In [ ]:
# Calculate the count of rows for each date
count_df = merged_df.groupby(['year', 'month', 'day'])['id'].count().reset_index()
count_df.rename(columns={'id': 'Obs'}, inplace=True)

# Merge the average data with the count data
result_df = pd.merge(result_df, count_df, on=['year', 'month', 'day'])

# Save the results to a new DataFrame
# result_df.to_csv('averages_by_date_with_count.csv', index=False)
result_df.to_csv('comments_liwc_average.csv', index=False, sep=',', quoting=csv.QUOTE_ALL, escapechar='\\')

## v2

In [5]:
import pandas as pd
import dask.dataframe as dd
from dask.delayed import delayed
import ast


In [6]:
# Load the preprocessed LIWC results from 'tmp-liwc.csv'
liwc_df = pd.read_csv('liwc-tmp.csv', sep=',', quoting=csv.QUOTE_ALL, escapechar='\\')

In [7]:
def remove_invalids(col='', df=''):
  invalid_options = [np.nan, 'nan', None, 0, '0', 'NaN', '[deleted]', '[removed]']
  df = df.dropna(subset=[col], how='all')
  df = df[~df[col].isin(invalid_options)]
  return df

# for source in reddit_files_titles:
source = 'argentina_comments'
source_dir = f'../output/reddit_output/{source}-output_table.csv'
source_output_dir = f'../output/reddit_output/filtered_tables_LIWC_count/{source}-liwc_output.csv'

In [8]:
df_output_table = pd.read_csv(source_dir, sep=',')

df_output_table = remove_invalids('text', df_output_table)

if 'title' in df_output_table.columns:  # if it's a submission (has title)
    # Remove invalid/incomplete observations
    df_output_table = remove_invalids('title', df_output_table)
    # Join title and text (fillna just in case something passed previous cleanup)
    df_output_table['text'] = df_output_table['title'].fillna('') + '\n' + df_output_table['text'].fillna('')
    # Now drop it
    df_output_table = df_output_table.drop(columns=['title'])


In [9]:
# Use Dask for the LIWC counts conversion
liwc_ddf = dd.from_pandas(liwc_df, npartitions=8)  # Adjust the number of partitions as needed

@delayed
def convert_to_dict(s):
    return ast.literal_eval(s)

liwc_ddf['text'] = liwc_ddf['text'].map(convert_to_dict, meta=('text', object)).compute()


In [11]:
import dask.dataframe as dd

# Merge the Dask DataFrames
df_merge = dd.merge(df_output_table, liwc_ddf, on='text')


: 

In [ ]:

# Save the merged DataFrame
df_merge.to_csv(source_output_dir, index=False, sep=',', quoting=csv.QUOTE_ALL, escapechar='\\')


## v1

In [ ]:
import pandas as pd
import json

# Load the preprocessed LIWC results from 'tmp-liwc.csv'
liwc_df = pd.read_csv('liwc-tmp.csv', sep=',', quoting=csv.QUOTE_ALL, escapechar='\\')


In [ ]:
import ast
liwc_df = pd.DataFrame(liwc_df['text'].apply(ast.literal_eval).to_list())

In [ ]:
def remove_invalids(col='', df=''):
  invalid_options = [np.nan, 'nan', None, 0, '0', 'NaN', '[deleted]', '[removed]']
  df = df.dropna(subset=[col], how='all')
  df = df[~df[col].isin(invalid_options)]
  return df

# for source in reddit_files_titles:
source = 'argentina_comments'
source_dir = f'../output/reddit_output/{source}-output_table.csv'
source_output_dir = f'../output/reddit_output/filtered_tables_LIWC_count/{source}-liwc_output.csv'

df_output_table = pd.read_csv(source_dir,sep=',')

df_output_table = remove_invalids('text', df_output_table)

if 'title' in df_output_table.columns: #if its a submission (has title)
    #remove invalid/incomplete obs
    df_output_table = remove_invalids('title', df_output_table)
    #join title and text (yes fillna just in case something passed previous cleanup)
    df_output_table['text'] = df_output_table['title'].fillna('') + '\n' + df_output_table['text'].fillna('')
    #now drop it
    df_output_table = df_output_table.drop(columns=['title'])

In [ ]:
df_merge_tst = pd.concat([df_output_table, liwc_df], axis=1)

In [ ]:
# Save the final DataFrame to a CSV
source_output_dir = f'../output/reddit_output/filtered_tables_LIWC_count/{source}-liwc_output.csv'
df_output_table.to_csv(source_output_dir, index=False, sep=',', quoting=csv.QUOTE_ALL, escapechar='\\')

In [8]:
sources = [
  'argentina_submissions',
  # 'argentina_comments',
  'Republica_Argentina_submissions',
  'Republica_Argentina_comments',
  'RepublicaArgentina_submissions',
  'RepublicaArgentina_comments',
]
for source in sources:
    df = pd.read_csv(f'/Volumes/Drakôn Kholkikos - 2TB/Tesis-Grado/output/reddit_output/filtered_tables_LIWC_count/{source}-liwc_output.csv', sep=',')
    print(source, '-', len(df))

/var/folders/fv/m61tb7rx6n5_86nwkz1zl_300000gn/T/ipykernel_32457/830965458.py:10: DtypeWarning: Columns (0,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'/Volumes/Drakôn Kholkikos - 2TB/Tesis-Grado/output/reddit_output/filtered_tables_LIWC_count/{source}-liwc_output.csv', sep=',')


argentina_submissions - 57645
Republica_Argentina_submissions - 2092


/var/folders/fv/m61tb7rx6n5_86nwkz1zl_300000gn/T/ipykernel_32457/830965458.py:10: DtypeWarning: Columns (0,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'/Volumes/Drakôn Kholkikos - 2TB/Tesis-Grado/output/reddit_output/filtered_tables_LIWC_count/{source}-liwc_output.csv', sep=',')


Republica_Argentina_comments - 172497
RepublicaArgentina_submissions - 5507
RepublicaArgentina_comments - 320714


/var/folders/fv/m61tb7rx6n5_86nwkz1zl_300000gn/T/ipykernel_32457/830965458.py:10: DtypeWarning: Columns (0,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'/Volumes/Drakôn Kholkikos - 2TB/Tesis-Grado/output/reddit_output/filtered_tables_LIWC_count/{source}-liwc_output.csv', sep=',')
